In [13]:
import pymysql
import pandas as pd

# 엑셀 파일 경로
excel_file = './data/ev_list.xlsx'

try:
    # 엑셀 파일 읽기
    df = pd.read_excel(excel_file)

    # '위도경도' 열에서 위도(latitude)와 경도(longitude)를 분리하는 함수 정의
    def split_lat_long(coord):
        if isinstance(coord, str):
            lat, longi = coord.split(',', 1)
            return pd.Series({'lat': lat.strip(), 'longi': longi.strip()})
        else:
            return pd.Series({'lat': None, 'longi': None})

    # '위도경도' 열을 분리하여 'lat'와 'longi' 열 추가
    df[['lat', 'longi']] = df['위도경도'].apply(split_lat_long)

    # 데이터 확인
    print("첫 5개 데이터 확인:\n", df.head(5))



원본 데이터프레임의 컬럼: Index(['설치년도', '시도', '군구', '주소', '충전소명', '시설구분(대)', '시설구분(소)', '기종(대)',
       '기종(소)', '운영기관(대)', '운영기관(소)', '급속충전량', '충전기타입', '이용자제한', '충전기ID',
       '충전소ID', '위도경도'],
      dtype='object')
첫 5개 데이터 확인:
    설치년도     시도   군구                 주소     충전소명 시설구분(대)  시설구분(소) 기종(대) 기종(소)  \
0  2020  서울특별시  종로구  서울특별시 종로구 종로3길 17  광화문 D타워    상업시설  마트(쇼핑몰)    완속  AC완속   
1  2020  서울특별시  종로구  서울특별시 종로구 종로3길 17  광화문 D타워    상업시설  마트(쇼핑몰)    완속  AC완속   
2  2020  서울특별시  종로구  서울특별시 종로구 종로3길 17  광화문 D타워    상업시설  마트(쇼핑몰)    완속  AC완속   
3  2020  서울특별시  종로구  서울특별시 종로구 종로3길 17  광화문 D타워    상업시설  마트(쇼핑몰)    완속  AC완속   
4  2020  서울특별시  종로구  서울특별시 종로구 종로3길 17  광화문 D타워    상업시설  마트(쇼핑몰)    완속  AC완속   

  운영기관(대) 운영기관(소) 급속충전량 충전기타입  이용자제한  충전기ID     충전소ID  \
0     타기관  휴맥스이브이   NaN  AC완속  이용자제한      3  HM000001   
1     타기관  휴맥스이브이   NaN  AC완속  이용자제한      4  HM000001   
2     타기관  휴맥스이브이   NaN  AC완속  이용자제한      5  HM000001   
3     타기관  휴맥스이브이   NaN  AC완속  이용자제한      6  HM000001   
4     타기관  휴

In [2]:
import pymysql
import pandas as pd

# 엑셀 파일 경로
excel_file = './data/ev_list.xlsx'

try:
    # 엑셀 파일 읽기
    df = pd.read_excel(excel_file)

    # 필요한 컬럼 선택
    df_selected = df[[ '군구', '주소', '충전소명', '충전소ID', '위도경도']]

    # '위도경도' 열에서 위도(latitude)와 경도(longitude)를 분리하여 새로운 열 추가
    def split_lat_long(coord):
        if isinstance(coord, str):
            lat, longi = coord.split(',', 1)
            return pd.Series({'lat': lat.strip(), 'longi': longi.strip()})
        else:
            return pd.Series({'lat': None, 'longi': None})

    df_selected[['lat', 'longi']] = df_selected['위도경도'].apply(split_lat_long)

    # 필요한 열 선택
    df_final = df_selected[[ '군구', '주소', '충전소명', '충전소ID', 'lat', 'longi']]

    # 중복된 행 제거
    df_final.drop_duplicates(inplace=True)

    # 위도와 경도가 null이 아닌 행만 선택
    df_final = df_final.dropna(subset=['lat', 'longi'])

    # MySQL 연결 설정
    conn = pymysql.connect(
        host='192.168.0.210',
        port=3306,
        user='seoulinfo',
        password='seoul',
        db='seoulinfo',
        charset='utf8'
    )
    
    # 커서 생성
    cursor = conn.cursor()

    # 데이터 삽입 (첫 5개 행만)
    for _, row in df_final.iterrows():
        # 데이터베이스에 삽입할 코드 작성
        sql = """
            INSERT INTO evc (
                evc_id, evc_area, evc_address, evc_name, 
                evc_lat, evc_long
            ) VALUES (%s, %s, %s, %s, %s, %s);
        """
        cursor.execute(sql, (
            row['충전소ID'], row['군구'], row['주소'], row['충전소명'], row['lat'], row['longi']
        ))

    # 변경사항 저장
    conn.commit()

    # 연결 닫기
    conn.close()
    print("첫 5개 데이터 DB 저장 완료")
except Exception as e:
    print('오류 발생:', e)


C:\Users\ict03_030\AppData\Local\Temp\ipykernel_2868\2070848921.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected[['lat', 'longi']] = df_selected['위도경도'].apply(split_lat_long)
C:\Users\ict03_030\AppData\Local\Temp\ipykernel_2868\2070848921.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected[['lat', 'longi']] = df_selected['위도경도'].apply(split_lat_long)
C:\Users\ict03_030\AppData\Local\Temp\ipykernel_2868\2070848921.py:28: SettingWithCopyWarning: 
A value is trying to be set on a c

첫 5개 데이터 DB 저장 완료


In [8]:
import pymysql
import pandas as pd

# 엑셀 파일 경로
excel_file = './data/ev_list.xlsx'

try:
    # 엑셀 파일 읽기
    df2 = pd.read_excel(excel_file)

    # 필요한 컬럼 선택
    df_selected2 = df2[[ '설치년도','시설구분(대)', '시설구분(소)', '기종(대)', '기종(소)','운영기관(대)', '운영기관(소)','충전기타입', '이용자제한','충전기ID','충전소ID']]

    print(df_selected2);
    
    # 필요한 열 선택
    df_final = df_selected2[['설치년도','시설구분(대)', '시설구분(소)', '기종(대)', '기종(소)','운영기관(대)', '운영기관(소)','충전기타입', '이용자제한','충전기ID','충전소ID']]

    # MySQL 연결 설정
    conn = pymysql.connect(
        host='192.168.0.210',
        port=3306,
        user='seoulinfo',
        password='seoul',
        db='seoulinfo',
        charset='utf8'
    )
    
    # 커서 생성
    cursor = conn.cursor()

    # 데이터 삽입 (첫 5개 행만)
    for _, row in df_final.head(50).iterrows():
        # 데이터베이스에 삽입할 코드 작성
        sql = """
            INSERT INTO evc_cg (
                charger_no, charger_status, charger_type, charger_userlimit, evc_id,
                charger_facbig, charger_facsmall,charger_mbig, charger_msmall,charger_opbig, charger_opsmall
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """
        cursor.execute(sql, (
            row['충전기ID'], row['이용자제한'], row['충전기타입'], row['이용자제한'], row['충전소ID'], row['시설구분(대)'], row['시설구분(소)'], row['기종(대)'], row['기종(소)'], row['운영기관(대)'] , row['운영기관(소)']
        ))

    # 변경사항 저장
    conn.commit()

    # 연결 닫기
    conn.close()
    print("첫 5개 데이터 DB 저장 완료")
except Exception as e:
    print('오류 발생:', e)


       설치년도 시설구분(대)  시설구분(소) 기종(대)            기종(소) 운영기관(대) 운영기관(소)  \
0      2020    상업시설  마트(쇼핑몰)    완속             AC완속     타기관  휴맥스이브이   
1      2020    상업시설  마트(쇼핑몰)    완속             AC완속     타기관  휴맥스이브이   
2      2020    상업시설  마트(쇼핑몰)    완속             AC완속     타기관  휴맥스이브이   
3      2020    상업시설  마트(쇼핑몰)    완속             AC완속     타기관  휴맥스이브이   
4      2020    상업시설  마트(쇼핑몰)    완속             AC완속     타기관  휴맥스이브이   
...     ...     ...      ...   ...              ...     ...     ...   
57083  2019    상업시설      주유소    급속  DC차데모+AC3상+DC콤보     타기관  evMost   
57084  2024    상업시설      주유소    급속             DC콤보     타기관      E1   
57085  2024    상업시설      주유소    급속             DC콤보     타기관      E1   
57086  2024    상업시설      주유소    급속             DC콤보     타기관      E1   
57087  2024    상업시설      주유소    급속             DC콤보     타기관      E1   

                 충전기타입  이용자제한  충전기ID     충전소ID  
0                 AC완속  이용자제한      3  HM000001  
1                 AC완속  이용자제한      4  HM000001  
